In [122]:
import re
import nltk
#
#Download nltk stopwords package if not present
#nltk.downlaod('stopwords')
#
#
import pandas as pd
import numpy as np

from nltk.corpus import stopwords

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer



class KaggleWord2VecUtility(object):


        @staticmethod
        def review_to_wordlist(review,stem=False):
            # Function to convert a document to a sequence of words,
            # optionally removing stop words.  Returns a list of words.
            #
            tknzr = TweetTokenizer()
            new_temp=[]
            review=str(review)
            new_temp=re.sub('\\<.*?\\>', "", review)
            new_temp=re.sub('\\http.*[\s]',"",new_temp)
            new_temp=re.sub('\\http.*$',"",new_temp)
            new_temp=re.sub('\\@(.*?)[\s]',"",new_temp)
            new_temp=re.sub('\\@(.*?)$',"",new_temp)
            #new_temp=re.sub('\\#(.*?)[\s]',"",new_temp)# if accuracy is less, try removing this
            #new_temp=re.sub('\\#(.*?)$',"",new_temp)# if accuracy is less, try removing this
            new_temp=re.sub(r'[^\w\s]',"",new_temp)

            review_text=re.sub('\d+', "",new_temp)#does this hamper performance

            words = review_text.lower().split()
            if  stem:
                ar1=[]
                ps=PorterStemmer()
                lanc=LancasterStemmer()
                lemma = nltk.wordnet.WordNetLemmatizer()
                sno=SnowballStemmer("english", ignore_stopwords=True)
                words = [sno.stem(w) for w in words]
                words = [ps.stem(w) for w in words]


            return(words)

In [123]:
import numpy as np
import pandas as pd
import re
import sklearn
from sklearn.svm import LinearSVC,SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,accuracy_score,recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from KaggleWord2VecUtility import KaggleWord2VecUtility
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import bigrams
from nltk.stem.porter import *
import xlrd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsRestClassifier
import sklearn
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN,BRkNNaClassifier,BRkNNbClassifier


#path = "D:\H\Dataset\Book_Dataset_clean.csv"
#df = pd.read_csv(path)
df = pd.read_excel('D:\H\Dataset\Book_Dataset_clean1.xlsx') #Chage to path where Excel file is stored

genre = df['Genre'][0:5000]  #Change values from [0:1000],[0:5000]or [] for 1000,5000,full dataset
summ = df["Summary"][0:5000] #Change values from [0:1000],[0:5000] or [] for 1000,5000,full dataset
result = []

#Regular Expressions

for xi in genre:
    abc = re.sub(r'"/[a-zA-Z/0-9 ]+": ',"",xi)
    abc = re.sub(r'[{}\"\"]+',"",abc)
    abc = re.sub(r'\\u00e0',"a",abc)
    abc = re.sub(r'/[a-zA-Z/0-9\_]+: ',"",abc)
    abc = re.sub(r', ',",",abc)
    result.append(abc)
    
genre_arr = np.array([np.array(xi.split(",")) for xi in result])


clean_summaries=[]
for i in range(0,len(summ)):
    clean_summaries.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(summ[i], True)))
clean_summaries = np.array(clean_summaries)




In [124]:
#Genre Replacing
arr2=[]
for i in genre_arr:
    arr3=[]
    for a in i:
        if a == 'Hard science fiction':
            b=a.replace('Hard science fiction','Science Fiction')
            arr3.append(b)
        elif a == 'Young adult literature':
            b=a.replace('Young adult literature',"Children's literature")
            arr3.append(b)
        elif a == 'Historical novel':
            b=a.replace('Historical novel','Historical')
            arr3.append(b)
        elif a == 'Historical fiction':
            b=a.replace('Historical fiction','Historical')
            arr3.append(b)
        elif a == 'Alternate history':
            b=a.replace('Alternate history','Historical')
            arr3.append(b)
        elif a == 'War novel':
            b=a.replace('War novel','Historical')
            arr3.append(b)
        elif a == 'Novella':
            b=a.replace('Novella','Novel')
            arr3.append(b)
        elif a == 'Comic novel':
            b=a.replace('Comic novel','Comic')
            arr3.append(b)
        elif a == 'Autobiographical novel':
            b=a.replace('Autobiographical novel','Autobiography')
            arr3.append(b)
        elif a == 'High fantasy':
            b=a.replace('High fantasy','Fantasy')
            arr3.append(b)
        elif a == 'History':
            b=a.replace('History','Historical')
            arr3.append(b)
        elif a == 'Memoir':
            b=a.replace('Memoir','Biography')
            arr3.append(b)
        elif a == 'Techno-thriller':
            b=a.replace('Techno-thriller','Thriller')
            arr3.append(b)
        elif a == 'Steampunk':
            b=a.replace('Steampunk','Science Fiction')
            arr3.append(b)
        elif a == 'Paranormal romance':
            b=a.replace('Paranormal romance','Romance')
            arr3.append(b)
        elif a == 'Romance novel':
            b=a.replace('Romance novel','Romance')
            arr3.append(b)
        elif a == 'Picture book':
            b=a.replace('Picture book',"Children's literature")
            arr3.append(b)
        elif a == 'Bildungsroman':
            b=a.replace('Bildungsroman','Non-fiction')
            arr3.append(b)
        elif a == 'Reference':
            b=a.replace('Reference','Non-fiction')
            arr3.append(b)
        elif a == 'Black comedy':
            b=a.replace('Black comedy','Comedy')
            arr3.append(b)
        elif a == 'Urban fantasy':
            b=a.replace('Urban fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Military science fiction':
            b=a.replace('Military science fiction','Science Fiction')
            arr3.append(b)
        elif a == 'Sword and sorcery':
            b=a.replace('Sword and sorcery','Speculative fiction')
            arr3.append(b)
        elif a == 'Whodunit':
            b=a.replace('Whodunit','Historical')
            arr3.append(b)
        elif a == 'Absurdist fiction':
            b=a.replace('Absurdist fiction','Literary fiction')
            arr3.append(b)
        elif a == 'Hardboiled':
            b=a.replace('Hardboiled','Detective fiction')
            arr3.append(b)
        elif a == 'Science':
            b=a.replace('Science','Science Fiction')
            arr3.append(b)
        elif a == 'Dark fantasy':
            b=a.replace('Dark fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Time travel':
            b=a.replace('Time travel','Adventure')
            arr3.append(b)
        elif a == 'Psychological novel':
            b=a.replace('Psychological novel','Fiction')
            arr3.append(b)
        elif a == 'Adventure novel':
            b=a.replace('Adventure novel','Adventure')
            arr3.append(b)
        elif a == 'Wuxia':
            b=a.replace('Wuxia','Fantasy')
            arr3.append(b)
        elif a == 'Picaresque novel':
            b=a.replace('Picaresque novel','Adventure')
            arr3.append(b)
        elif a == 'True crime':
            b=a.replace('True crime','Non-fiction')
            arr3.append(b)
        elif a == 'Cyberpunk':
            b=a.replace('Cyberpunk','Science Fiction')
            arr3.append(b)
        elif a == 'Historical fantasy':
            b=a.replace('Historical fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Vampire fiction':
            b=a.replace('Vampire fiction','Gothic fiction')
            arr3.append(b)
        elif a == 'Religion':
            b=a.replace('Religion','Philosophy')
            arr3.append(b)
        elif a == 'Scientific romance':
            b=a.replace('Scientific romance','Romance')
            arr3.append(b)
        elif a == 'Lost World':
            b=a.replace('Lost World','Science Fiction')
            arr3.append(b)
        elif a == 'Space opera':
            b=a.replace('Space opera','Science Fiction')
            arr3.append(b)
        elif a == 'Chivalric romance':
            b=a.replace('Chivalric romance','Romance')
            arr3.append(b)
        elif a == 'K\\u00fcnstlerroman':
            b=a.replace('K\\u00fcnstlerroman','Kunstlerroman')
            arr3.append(b)
        elif a == 'Biographical novel':
            b=a.replace('Biographical novel','Biography')
            arr3.append(b)
        elif a == 'Travel literature':
            b=a.replace('Travel literature','Adventure')
            arr3.append(b)
        elif a == 'Science fantasy':
            b=a.replace('Science fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Comic book':
            b=a.replace('Comic book','Comic')
            arr3.append(b)
        elif a == 'Kunstlerroman':
            b=a.replace('Kunstlerroman','Non-fiction')
            arr3.append(b)
        elif a == 'Drama':
            b=a.replace('Drama','Fiction')
            arr3.append(b)
        elif a == 'Parody':
            b=a.replace('Parody','Satire')
            arr3.append(b)
        elif a == 'Travel':
            b=a.replace('Travel','Adventure')
            arr3.append(b)
        elif a == 'Magic realism':
            b=a.replace('Magic realism','Fantasy')
            arr3.append(b)
        elif a == 'Epistolary novel':
            b=a.replace('Epistolary novel','Non-fiction')
            arr3.append(b)
       
        elif a == 'Politics':
            b=a.replace('Politics','Fiction')
            arr3.append(b)
        elif a == 'Mathematics':
            b=a.replace('Mathematics','Non-fiction')
            arr3.append(b)
        elif a == 'Marketing':
            b=a.replace('Marketing','Non-fiction')
            arr3.append(b)
        elif a == 'American Gothic Fiction':
            b=a.replace('American Gothic Fiction','Gothic fiction')
            arr3.append(b)
        elif a == 'Police procedural':
            b=a.replace('Police procedural','Detective fiction')
            arr3.append(b)
        elif a == 'Human extinction':
            b=a.replace('Human extinction','Apocalyptic and post-apocalyptic fiction')
            arr3.append(b)
        elif a == 'Utopian fiction':
            b=a.replace('Utopian fiction','Utopian and dystopian fiction')
            arr3.append(b)
        elif a == 'Photography':
            b=a.replace('Photography','Non-fiction')
            arr3.append(b)
        elif a == 'Future history':
            b=a.replace('Future history','Speculative fiction')
            arr3.append(b)
        elif a == 'Fantastique':
            b=a.replace('Fantastique','Fantasy')
            arr3.append(b)
        elif a == 'Popular culture':
            b=a.replace('Popular culture','Non-fiction')
            arr3.append(b)
        elif a == 'Robinsonade':
            b=a.replace('Robinsonade','Travel')
            arr3.append(b)
        elif a == 'Conspiracy':
            b=a.replace('Conspiracy','Thriller')
            arr3.append(b)
        elif a == 'Colonial United States romance':
            b=a.replace('Colonial United States romance','Romance')
            arr3.append(b)
        elif a == 'Experimental literature':
            b=a.replace('Experimental literature','Fiction')
            arr3.append(b)
        elif a == 'Religious text':
            b=a.replace('Religious text','Philosophy')
            arr3.append(b)
        elif a == 'English public-school stories':
            b=a.replace('English public-school stories','Fiction')
            arr3.append(b)       
        elif a == 'School story':
            b=a.replace('School story','Fiction')
            arr3.append(b)
        elif a == "Boys' school stories":
            b=a.replace("Boys' school stories",'Fiction')
            arr3.append(b)
        elif a == 'Social criticism':
            b=a.replace('Social criticism','Non-fiction')
            arr3.append(b)
        elif a == 'Computer Science':
            b=a.replace('Computer Science','Non-fiction')
            arr3.append(b)
        elif a == 'Pornography':
            b=a.replace('Pornography','Fantasy')
            arr3.append(b)
        elif a == 'Bit Lit':
            b=a.replace('Bit Lit','Fantasy')
            arr3.append(b)
        elif a == 'Georgian romance':
            b=a.replace('Georgian romance','Romance')
            arr3.append(b)
        elif a == 'Superhero fiction':
            b=a.replace('Superhero fiction','Fiction')
            arr3.append(b)
        elif a == 'New Weird':
            b=a.replace('New Weird','Speculative fiction')
            arr3.append(b)
        elif a == 'Alien invasion':
            b=a.replace('Alien invasion','Science Fiction')
            arr3.append(b)
        elif a == 'Low fantasy':
            b=a.replace('Low fantasy','Fantasy')
            arr3.append(b)
        elif a == 'New York Times Best Seller list':
            b=a.replace('New York Times Best Seller list','Non-fiction')
            arr3.append(b)
        elif a == 'Western fiction':
            b=a.replace('Western fiction','Western')
            arr3.append(b)
        elif a == 'Neuroscience':
            b=a.replace('Neuroscience','Non-fiction')
            arr3.append(b)
        elif a == 'Juvenile fantasy':
            b=a.replace('Juvenile fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Medieval romance':
            b=a.replace('Medieval romance','Romance')
            arr3.append(b)
        elif a == 'Romantic comedy':
            b=a.replace('Romantic comedy','Comedy')
            arr3.append(b)
        elif a == 'Foreign legion':
            b=a.replace('Foreign legion','Adventure')
            arr3.append(b)
        elif a == 'Fable':
            b=a.replace('Fable','Fiction')
            arr3.append(b)
        elif a == 'Elizabethan romance':
            b=a.replace('Elizabethan romance','Romance')
            arr3.append(b)
        elif a == 'Metaphysics':
            b=a.replace('Metaphysics','Philosophy')
            arr3.append(b)
        elif a == 'Bangsian fantasy':
            b=a.replace('Bangsian fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Indian chick lit':
            b=a.replace('Indian chick lit','Fiction')
            arr3.append(b)
        elif a == 'Epic Science Fiction and Fantasy':
            b=a.replace('Epic Science Fiction and Fantasy','Science Fiction')
            arr3.append(b)
        elif a == 'Naval adventure':
            b=a.replace('Naval adventure','Adventure')
            arr3.append(b)
        elif a == 'Cookbook':
            b=a.replace('Cookbook','Non-fiction')
            arr3.append(b)
        elif a == 'Albino bias':
            b=a.replace('Albino bias','Fiction')
            arr3.append(b)
        elif a == 'Morality play':
            b=a.replace('Morality play','Drama')
            arr3.append(b)
        elif a == 'Polemic':
            b=a.replace('Polemic','Fiction')
            arr3.append(b)
        elif a == 'Literary criticism':
            b=a.replace('Literary criticism','Non-fiction')
            arr3.append(b)
        elif a == 'Creative nonfiction':
            b=a.replace('Creative nonfiction','Non-fiction')
            arr3.append(b)
        elif a == 'Postcyberpunk':
            b=a.replace('Postcyberpunk','Science Fiction')
            arr3.append(b)
        elif a == 'Fairytale fantasy':
            b=a.replace('Fairytale fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Modernism':
            b=a.replace('Modernism','Non-fiction')
            arr3.append(b)
        elif a == 'Political philosophy':
            b=a.replace('Political philosophy','Philosophy')
            arr3.append(b)
        elif a == 'Anti-war':
            b=a.replace('Anti-war','Historical')
            arr3.append(b)
        elif a == 'Gay Themed':
            b=a.replace('Gay Themed','Speculative fiction')
            arr3.append(b)
        elif a == 'Military history':
            b=a.replace('Military history','Non-fiction')
            arr3.append(b)
        elif a == 'First-person narrative':
            b=a.replace('First-person narrative','Fiction')
            arr3.append(b)
        elif a == 'Fictional crossover':
            b=a.replace('Fictional crossover','Fiction')
            arr3.append(b)
        elif a == 'Space western':
            b=a.replace('Space western','Western')
            arr3.append(b)
        elif a == 'Social novel':
            b=a.replace('Social novel','Fiction')
            arr3.append(b)
        elif a == 'Heroic fantasy':
            b=a.replace('Heroic fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Sea story':
            b=a.replace('Sea story','Fiction')
            arr3.append(b)
        elif a == 'Coming of age':
            b=a.replace('Coming of age','Non-fiction')
            arr3.append(b)
        elif a == 'Self-help':
            b=a.replace('Self-help','Non-fiction')
            arr3.append(b)
        elif a == 'Catastrophic literature':
            b=a.replace('Catastrophic literature','Apocalyptic and post-apocalyptic fiction')
            arr3.append(b)
        elif a == 'Dying Earth subgenre':
            b=a.replace('Dying Earth subgenre','Science Fiction')
            arr3.append(b)
        elif a == 'Zombies in popular culture':
            b=a.replace('Zombies in popular culture','Fiction')
            arr3.append(b)
        elif a == 'Social sciences':
            b=a.replace('Social sciences','Science Fiction')
            arr3.append(b)
        elif a == 'Treatise':
            b=a.replace('Treatise','Non-fiction')
            arr3.append(b)
        elif a == 'Essay':
            b=a.replace('Essay','Non-fiction')
            arr3.append(b)
        elif a == 'Prose poetry':
            b=a.replace('Prose poetry','Prose')
            arr3.append(b)
        elif a == 'Ergodic literature':
            b=a.replace('Ergodic literature','Fiction')
            arr3.append(b)
        elif a == 'Non-fiction novel':
            b=a.replace('Non-fiction novel','Non-fiction')
            arr3.append(b)
        elif a == 'LGBT literature':
            b=a.replace('LGBT literature','Speculative fiction')
            arr3.append(b)
        elif a == 'Transgender and transsexual fiction':
            b=a.replace('Transgender and transsexual fiction','Speculative fiction')
            arr3.append(b)
        elif a == 'Social commentary':
            b=a.replace('Social commentary','Non-fiction')
            arr3.append(b)
        elif a == 'Sword and planet':
            b=a.replace('Sword and planet','Fantasy')
            arr3.append(b)
        elif a == 'Biopunk':
            b=a.replace('Biopunk','Science Fiction')
            arr3.append(b)
        elif a == 'Comedy of manners':
            b=a.replace('Comedy of manners','Comedy')
            arr3.append(b)
        elif a == 'Locked room mystery':
            b=a.replace('Locked room mystery','Mystery')
            arr3.append(b)
        elif a == 'Industrial novel':
            b=a.replace('Industrial novel','Fiction')
            arr3.append(b)
        elif a == 'Industrial novel':
            b=a.replace('Industrial novel','Fiction')
            arr3.append(b)
        elif a == 'Mashup':
            b=a.replace('Mashup','Fiction')
            arr3.append(b)
        elif a == 'Edisonade':
            b=a.replace('Edisonade','Science Fiction')
            arr3.append(b)
        elif a == 'Farce':
            b=a.replace('Farce','Comedy')
            arr3.append(b)
        elif a == 'Historical romance':
            b=a.replace('Historical romance','Romance')
            arr3.append(b)
        elif a == 'Conspiracy fiction':
            b=a.replace('Conspiracy fiction','Thriller')
            arr3.append(b)
        elif a == 'Spirituality':
            b=a.replace('Spirituality','Philosophy')
            arr3.append(b)
        elif a == 'Zombie':
            b=a.replace('Zombie','Fiction')
            arr3.append(b)
        elif a == 'Subterranean fiction':
            b=a.replace('Subterranean fiction','Fiction')
            arr3.append(b)
        elif a == 'Ghost story':
            b=a.replace('Ghost story','Horror')
            arr3.append(b)
        elif a == 'Supernatural':
            b=a.replace('Supernatural','Horror')
            arr3.append(b)
        elif a == 'Anthropology':
            b=a.replace('Anthropology','Historical')
            arr3.append(b)
        elif a == 'Sports':
            b=a.replace('Sports','Non-fiction')
            arr3.append(b)
        elif a == 'Post-holocaust':
            b=a.replace('Post-holocaust','Historical')
            arr3.append(b)
        elif a == 'Comic fantasy':
            b=a.replace('Comic fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Planetary romance':
            b=a.replace('Planetary romance','Romance')
            arr3.append(b)
        elif a == 'Comics':
            b=a.replace('Comics','Comic')
            arr3.append(b)
        elif a == 'Chick lit':
            b=a.replace('Chick lit','Fiction')
            arr3.append(b)
        elif a == 'Cozy':
            b=a.replace('Cozy','Crime Fiction')
            arr3.append(b)
        elif a == 'Inspirational':
            b=a.replace('Inspirational','Non-fiction')
            arr3.append(b)
        elif a == 'Fantasy of manners':
            b=a.replace('Fantasy of manners','Fantasy')
            arr3.append(b)
        elif a == 'Anthology':
            b=a.replace('Anthology','Short story')
            arr3.append(b)
        elif a == 'Contemporary fantasy':
            b=a.replace('Contemporary fantasy','Fantasy')
            arr3.append(b)
        elif a == 'Transhumanism':
            b=a.replace('Transhumanism','Science Fiction')
            arr3.append(b)
        elif a == 'Campus novel':
            b=a.replace('Campus novel','Non-fiction')
            arr3.append(b)
        elif a == 'Soft science fiction':
            b=a.replace('Soft science fiction','Science Fiction')
            arr3.append(b)
        elif a == 'Comic fantasy':
            b=a.replace('Comic fantasy','Comic')
            arr3.append(b)
        elif a == 'Urban fiction':
            b=a.replace('Urban fiction','Fiction')
            arr3.append(b)
        elif a == 'Social science fiction':
            b=a.replace('Social science fiction','Fiction')
            arr3.append(b)
        elif a == 'Economics':
            b=a.replace('Economics','Non-fiction')
            arr3.append(b)
        elif a == 'Erotica':
            b=a.replace('Erotica','Fiction')
            arr3.append(b)
        elif a == 'Graphic novel':
            b=a.replace('Graphic novel','Fiction')
            arr3.append(b)
        elif a == 'Regency romance':
            b=a.replace('Regency romance','Romance')
            arr3.append(b)
        elif a == 'Tragicomedy':
            b=a.replace('Tragicomedy','Comedy')
            arr3.append(b)
        elif a == 'Fairy tale':
            b=a.replace('Fairy tale','Fantasy')
            arr3.append(b)
        elif a == 'Literary realism':
            b=a.replace('Literary realism','Non-fiction')
            arr3.append(b)
        elif a == 'Existentialism':
            b=a.replace('Existentialism','Philosophy')
            arr3.append(b)
        elif a == 'Popular science':
            b=a.replace('Popular science','Non-fiction')
            arr3.append(b)
        elif a == 'Feminist science fiction':
            b=a.replace('Feminist science fiction','Science Fiction')
            arr3.append(b)
        elif a == 'Postmodernism':
            b=a.replace('Postmodernism','Non-fiction')
            arr3.append(b)
        elif a == 'Personal journal':
            b=a.replace('Personal journal','Non-fiction')
            arr3.append(b)
        elif a == 'Gay novel':
            b=a.replace('Gay novel','Speculative fiction')
            arr3.append(b)
        elif a == 'Comic science fiction':
            b=a.replace('Comic science fiction','Science Fiction')
            arr3.append(b)
        elif a == 'Historical whodunnit':
            b=a.replace('Historical whodunnit','Historical')
            arr3.append(b)
        elif a == 'Psychology':
            b=a.replace('Psychology','Fiction')
            arr3.append(b)
        elif a == 'Dystopia':
            b=a.replace('Dystopia','Utopian and dystopian fiction')
            arr3.append(b)
        elif a == 'Sociology':
            b=a.replace('Sociology','Non-fiction')
            arr3.append(b)
        else:
            arr3.append(a)
    arr2.append(arr3)


In [125]:
import itertools
from collections import Counter

array1=[]
totals = Counter(i for i in list(itertools.chain.from_iterable(arr2))) #Dictionary for count of each genre

print(totals)
print(len(totals))

for genre in totals:
    if totals.get(genre)<=100:  #set values as 25,100,200 to reduce the count of genres
        array1.append(genre)
array3=[a for a in totals]

array2=([[genre for genre in i if genre not in array1]for i in arr2])

print(len(genre_arr))  #Checking if any instance has a blank target label set.
print(len(array2))     #

res = Counter(i for i in list(itertools.chain.from_iterable(array2)))
print (res)          #reduced Genre set




Counter({'Fiction': 2651, 'Speculative fiction': 2225, 'Science Fiction': 1443, 'Novel': 1208, "Children's literature": 1114, 'Fantasy': 1016, 'Historical': 615, 'Mystery': 483, 'Suspense': 360, 'Horror': 287, 'Crime Fiction': 244, 'Thriller': 217, 'Non-fiction': 216, 'Romance': 192, 'Adventure': 146, 'Utopian and dystopian fiction': 115, 'Autobiography': 103, 'Comedy': 103, 'Detective fiction': 99, 'Spy fiction': 86, 'Biography': 82, 'Satire': 78, 'Gothic fiction': 69, 'Comic': 62, 'Philosophy': 48, 'Humour': 44, 'Short story': 36, 'Apocalyptic and post-apocalyptic fiction': 35, 'Gamebook': 35, 'Western': 31, 'Literary fiction': 30, 'Roman a clef': 21, 'Poetry': 15, 'Role-playing game': 13, 'Kunstlerroman': 10, 'Parallel novel': 10, 'Business': 7, 'Music': 5, 'Invasion literature': 4, 'Nature': 4, 'Drama': 2, 'Serial': 2, 'Play': 2, 'Prose': 1, 'Anti-nuclear': 1, 'Cabal': 1})
46
5000
5000
Counter({'Fiction': 2651, 'Speculative fiction': 2225, 'Science Fiction': 1443, 'Novel': 1208, "C

In [126]:
#MultiLabelBinarizer

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(array2)


In [127]:
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation,Flatten
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras import metrics
from keras import backend as K


b= np_utils.to_categorical(Y,3)

tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(clean_summaries)
sequences = tokenizer.texts_to_sequences(clean_summaries)
data = pad_sequences(sequences, maxlen=500)

X_train, X_test, y_train, y_test = train_test_split( data, Y, test_size=0.2, random_state=42)

print (X_train.shape)

model = Sequential()
model.add(Embedding(20000, 64, input_length=500))
model.add(Dropout(0.3))
model.add(Conv1D(64, 9, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(16, activation='sigmoid'))  #Chanage the dense layer attribute value as indicated by error msg. it changes when count of genres is chamged.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train,epochs=10,validation_split=0.2,batch_size=10)



             precision    recall  f1-score   support

          0       0.60      0.02      0.04       135
          1       0.00      0.00      0.00       100
          2       0.61      0.24      0.35      1000
          3       0.00      0.00      0.00        98
          4       0.62      0.17      0.26       233
          5       0.74      0.44      0.55       979
          6       0.59      0.67      0.63      2611
          7       0.74      0.22      0.34       504
          8       0.67      0.20      0.30       284
          9       0.63      0.25      0.36       480
         10       0.54      0.07      0.12       196
         11       0.49      0.14      0.22      1204
         12       0.00      0.00      0.00       181
         13       0.81      0.60      0.69      1369
         14       0.75      0.69      0.72      2221
         15       0.54      0.10      0.17       360
         16       0.54      0.06      0.11       204
         17       0.00      0.00      0.00   

c:\users\shrey\appdata\local\programs\python\python35\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [128]:
from sklearn.metrics import coverage_error
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from scipy.sparse import lil_matrix
pred=model.predict(X_test)

result=[]

'''Since the neural networks ouput an array of probabilities hence these need to be converted to binary form for inverse transforming them.
A threshold value of 0.18 is set. All probabilities below 0.18 are set to 0 and all others to 1
'''
for i in pred:
    #print (i)
    tempx=[]
    for a in i:
        if a<0.18:
            a=0;
            tempx.append(a);
        else:
            a=1;
            tempx.append(a);
    result.append(tempx)
#result=np.array(result)

result = lil_matrix(result)


print (coverage_error(y_test,pred))
result1=mlb.inverse_transform(result)
actual=mlb.inverse_transform(y_test)


res = classification_report(y_test,result)
print(res)

from sklearn.metrics import hamming_loss     #Hamming_loss is calculated

print(hamming_loss(Y,y_pred))



0.10622222222222222
